# 📈 Curvas de Respuesta y Optimización de Presupuesto

**Objetivo**: Ajustar curvas de respuesta Hill a la atribución incremental de cada canal y optimizar la asignación de presupuesto.

## ¿Por qué es crítico tener variables limpias?

Si el modelo incluía `clicks_META` e `impressions_META` juntas, el efecto del canal se repartía entre ambas. Al ajustar una curva solo con `impressions_META`, **subestimaríamos** masivamente el impacto real.

**Ahora**, con variables limpias:
- Todo el efecto de META está en `impressions_META`
- Todo el efecto de GADS está en `impressions_GADS`
- Las curvas reflejan el **impacto total** del canal

**Inputs**:
- `atribucion_incremental.csv` (del notebook 2)
- `dataset_limpio_sin_multicolinealidad.csv` (del notebook 1 - para impressions)


## 1. Setup


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, minimize
import warnings

warnings.filterwarnings('ignore')

print("✓ Librerías cargadas")


In [ ]:
# Cargar atribución incremental (output del notebook 2)
attrib = pd.read_csv("atribucion_incremental.csv")
attrib["Fecha"] = pd.to_datetime(attrib["Fecha"])

print(f"Atribución cargada: {attrib.shape}")
print(f"  Empresas: {attrib['empresa'].nunique()}")
print(f"  Período: {attrib['Fecha'].min()} → {attrib['Fecha'].max()}")

# Cargar dataset limpio (para obtener impressions originales)
df_clean = pd.read_csv("dataset_limpio_sin_multicolinealidad.csv")
df_clean["Fecha"] = pd.to_datetime(df_clean.get("Fecha", df_clean.get("week_start")))

print(f"\\n Dataset limpio cargado: {df_clean.shape}")

# Verificar que las variables auxiliares para optimización estén disponibles
AUX_VARS_NEEDED = ["invest_META", "invest_GADS", "cpm_META", "cpm_GADS"]
aux_vars_present = [v for v in AUX_VARS_NEEDED if v in df_clean.columns]

print(f"\n🔧 Variables auxiliares para optimización:")
for var in AUX_VARS_NEEDED:
    status = "✅" if var in df_clean.columns else "❌"
    print(f"  {status} {var}")

if len(aux_vars_present) < len(AUX_VARS_NEEDED):
    missing = [v for v in AUX_VARS_NEEDED if v not in df_clean.columns]
    print(f"\n⚠️ ADVERTENCIA: Faltan variables para optimización: {missing}")
    print("   Las funciones de optimización de presupuesto pueden no funcionar correctamente.")
else:
    print(f"\n✅ Todas las variables auxiliares disponibles para optimización")


## 2. Preparar Dataset para Curvas

Unimos la atribución incremental con las impresiones originales.


In [ ]:
# Seleccionar columnas necesarias del dataset limpio
keys = ["Fecha", "empresa"]
media_cols = [c for c in ["impressions_META", "impressions_GADS"] if c in df_clean.columns]

if len(media_cols) == 0:
    print("⚠️ ERROR: No se encontraron columnas de impressions en el dataset limpio")
    print(f"   Columnas disponibles: {df_clean.columns.tolist()}")
else:
    df_media = df_clean[keys + media_cols].copy()
    
    # Merge con atribución
    df_curve = attrib.merge(df_media, on=keys, how="inner")
    df_curve = df_curve.dropna()
    
    print(f"\\n✓ Dataset para curvas preparado: {df_curve.shape}")
    print(f"\\nColumnas disponibles:")
    print(f"  • impressions_META: {'✅' if 'impressions_META' in df_curve.columns else '❌'}")
    print(f"  • impressions_GADS: {'✅' if 'impressions_GADS' in df_curve.columns else '❌'}")
    print(f"  • META_incr: {'✅' if 'META_incr' in df_curve.columns else '❌'}")
    print(f"  • GADS_incr: {'✅' if 'GADS_incr' in df_curve.columns else '❌'}")
    
    print(f"\\nResumen:")
    print(df_curve[[c for c in ['impressions_META', 'impressions_GADS', 'META_incr', 'GADS_incr'] if c in df_curve.columns]].describe())


## 3. Función Hill y Ajuste

La curva Hill modela saturación: `T(x) = β · (x^α / (k^α + x^α))`

Donde:
- `x`: Input (impressions)
- `T(x)`: Output (transacciones incrementales)
- `α (alpha)`: Forma de la curva (>1 = sigmoide, <1 = convexa)
- `k`: Punto medio de saturación
- `β (beta)`: Máximo asintótico


In [ ]:
def hill(x, alpha, k):
    """Curva de saturación Hill"""
    x = np.clip(np.asarray(x, float), 0, None)
    alpha = max(float(alpha), 1e-8)
    k = max(float(k), 1e-8)
    return np.power(x, alpha) / (np.power(k, alpha) + np.power(x, alpha))

def hill_scaled(x, alpha, k, beta):
    """Curva Hill escalada por beta"""
    beta = max(float(beta), 1e-12)
    return beta * hill(x, alpha, k)

def fit_hill(x, y, alpha0=1.2, k0=None, beta0=None,
             bounds_alpha=(0.3, 5.0), bounds_k=(1e-6, 1e12), bounds_beta=(1e-6, 1e12)):
    """
    Ajusta curva Hill a datos (x, y)
    
    Returns:
        dict con 'alpha', 'k', 'beta', 'r2', 'y_hat', 'success'
    """
    x = np.asarray(x, float)
    y = np.clip(np.asarray(y, float), 0, None)
    
    # Valores iniciales
    if k0 is None:
        k0 = np.median(x[x > 0]) if np.any(x > 0) else 1.0
    if beta0 is None:
        beta0 = max(np.nanmax(y) if np.isfinite(np.nanmax(y)) else 1.0, 1.0)
    
    p0 = np.array([alpha0, float(k0), float(beta0)], dtype=float)
    lb = np.array([bounds_alpha[0], bounds_k[0], bounds_beta[0]], dtype=float)
    ub = np.array([bounds_alpha[1], bounds_k[1], bounds_beta[1]], dtype=float)
    
    # Residuos
    def resid(p):
        a, k, b = p
        return hill_scaled(x, a, k, b) - y
    
    # Optimización
    res = least_squares(resid, p0, bounds=(lb, ub), loss="soft_l1", method="trf")
    a, k, b = map(float, res.x)
    
    # Predicciones y R²
    y_hat = hill_scaled(x, a, k, b)
    ss_res = float(np.sum((y - y_hat)**2))
    ss_tot = float(np.sum((y - y.mean())**2)) + 1e-12
    r2 = 1.0 - ss_res/ss_tot
    
    return {
        "alpha": a, 
        "k": k, 
        "beta": b, 
        "r2": r2, 
        "y_hat": y_hat, 
        "success": bool(res.success)
    }

def hill_marginal(x, alpha, k, beta):
    """Retorno marginal: dT/dx"""
    x = np.clip(np.asarray(x, float), 1e-12, None)
    a, kval, b = float(alpha), float(k), float(beta)
    num = b * a * (kval**a) * (x**(a-1))
    den = (kval**a + x**a)**2
    return num / den

print("✓ Funciones de curva Hill definidas")


In [ ]:
# Ajustar curva global para META
if "impressions_META" in df_curve.columns and "META_incr" in df_curve.columns:
    d_meta = df_curve[["impressions_META", "META_incr"]].dropna()
    res_META = fit_hill(d_meta["impressions_META"].values, d_meta["META_incr"].values, alpha0=1.2)
    
    print("\\n📈 CURVA META (global):")
    print(f"  α (alpha) = {res_META['alpha']:.4f} (forma de curva)")
    print(f"  k         = {res_META['k']:,.1f} (punto medio de saturación)")
    print(f"  β (beta)  = {res_META['beta']:.4f} (máximo asintótico)")
    print(f"  R²        = {res_META['r2']:.4f}")
    print(f"  Converged = {res_META['success']}")
else:
    print("⚠️ No se pudo ajustar curva META")
    res_META = None

# Ajustar curva global para GADS
if "impressions_GADS" in df_curve.columns and "GADS_incr" in df_curve.columns:
    d_gads = df_curve[["impressions_GADS", "GADS_incr"]].dropna()
    res_GADS = fit_hill(d_gads["impressions_GADS"].values, d_gads["GADS_incr"].values, alpha0=1.1)
    
    print("\\n📈 CURVA GADS (global):")
    print(f"  α (alpha) = {res_GADS['alpha']:.4f}")
    print(f"  k         = {res_GADS['k']:,.1f}")
    print(f"  β (beta)  = {res_GADS['beta']:.4f}")
    print(f"  R²        = {res_GADS['r2']:.4f}")
    print(f"  Converged = {res_GADS['success']}")
else:
    print("⚠️ No se pudo ajustar curva GADS")
    res_GADS = None


## 5. Visualización de Curvas


In [ ]:
if res_META is not None and res_GADS is not None:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # META
    x_meta = d_meta["impressions_META"].values
    y_meta = d_meta["META_incr"].values
    x_range_meta = np.linspace(0, x_meta.max()*1.1, 300)
    y_pred_meta = hill_scaled(x_range_meta, res_META["alpha"], res_META["k"], res_META["beta"])
    
    ax1.scatter(x_meta, y_meta, alpha=0.3, s=10, label="Datos reales")
    ax1.plot(x_range_meta, y_pred_meta, 'r-', linewidth=2, label="Curva Hill ajustada")
    ax1.axvline(res_META["k"], color='orange', linestyle='--', alpha=0.7, label=f"k={res_META['k']:,.0f}")
    ax1.set_xlabel("Impressions META")
    ax1.set_ylabel("Transacciones Incrementales")
    ax1.set_title(f"Curva de Respuesta META (R²={res_META['r2']:.3f})", fontsize=14, weight="bold")
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # GADS
    x_gads = d_gads["impressions_GADS"].values
    y_gads = d_gads["GADS_incr"].values
    x_range_gads = np.linspace(0, x_gads.max()*1.1, 300)
    y_pred_gads = hill_scaled(x_range_gads, res_GADS["alpha"], res_GADS["k"], res_GADS["beta"])
    
    ax2.scatter(x_gads, y_gads, alpha=0.3, s=10, label="Datos reales", color="green")
    ax2.plot(x_range_gads, y_pred_gads, 'r-', linewidth=2, label="Curva Hill ajustada")
    ax2.axvline(res_GADS["k"], color='orange', linestyle='--', alpha=0.7, label=f"k={res_GADS['k']:,.0f}")
    ax2.set_xlabel("Impressions GADS")
    ax2.set_ylabel("Transacciones Incrementales")
    ax2.set_title(f"Curva de Respuesta GADS (R²={res_GADS['r2']:.3f})", fontsize=14, weight="bold")
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\\n✅ Curvas visualizadas correctamente")
else:
    print("⚠️ No se pudieron visualizar las curvas (faltan ajustes)")


## 6. Análisis de Retorno Marginal


In [ ]:
if res_META is not None and res_GADS is not None:
    # Nivel actual (mediana)
    x_meta_current = float(d_meta["impressions_META"].median())
    x_gads_current = float(d_gads["impressions_GADS"].median())
    
    # Retornos marginales actuales
    marg_meta = hill_marginal(x_meta_current, res_META["alpha"], res_META["k"], res_META["beta"])
    marg_gads = hill_marginal(x_gads_current, res_GADS["alpha"], res_GADS["k"], res_GADS["beta"])
    
    print(f"\\n📊 RETORNO MARGINAL (en nivel actual de actividad):")
    print(f"  META: {marg_meta:.6f} transacciones / impresión")
    print(f"  GADS: {marg_gads:.6f} transacciones / impresión")
    
    # Ejemplo con CPM (costo por 1000 impressions)
    # Ajusta estos valores según tus costos reales
    CPM_META = 2.5  # USD por 1000 impressions
    CPM_GADS = 3.2  # USD por 1000 impressions
    
    imp_por_dolar_meta = 1000.0 / CPM_META
    imp_por_dolar_gads = 1000.0 / CPM_GADS
    
    marg_por_dolar_meta = marg_meta * imp_por_dolar_meta
    marg_por_dolar_gads = marg_gads * imp_por_dolar_gads
    
    print(f"\\n💰 RETORNO MARGINAL POR DÓLAR (asumiendo CPM={CPM_META} META, {CPM_GADS} GADS):")
    print(f"  META: {marg_por_dolar_meta:.6f} transacciones / USD")
    print(f"  GADS: {marg_por_dolar_gads:.6f} transacciones / USD")
    
    if marg_por_dolar_gads > marg_por_dolar_meta:
        print(f"\\n  → GADS es {marg_por_dolar_gads/marg_por_dolar_meta:.2f}x más eficiente en el margen")
    else:
        print(f"\\n  → META es {marg_por_dolar_meta/marg_por_dolar_gads:.2f}x más eficiente en el margen")


## 7. Exportar Curvas


In [ ]:
if res_META is not None and res_GADS is not None:
    # Guardar parámetros de las curvas
    curves_params = pd.DataFrame({
        "canal": ["META", "GADS"],
        "alpha": [res_META["alpha"], res_GADS["alpha"]],
        "k": [res_META["k"], res_GADS["k"]],
        "beta": [res_META["beta"], res_GADS["beta"]],
        "r2": [res_META["r2"], res_GADS["r2"]],
    })
    curves_params.to_csv("parametros_curvas_hill.csv", index=False)
    print("✓ Guardado: 'parametros_curvas_hill.csv'")
    
    # Guardar predicciones (para validación)
    df_curve_out = df_curve.copy()
    df_curve_out["META_pred"] = hill_scaled(
        df_curve_out["impressions_META"].values, 
        res_META["alpha"], res_META["k"], res_META["beta"]
    )
    df_curve_out["GADS_pred"] = hill_scaled(
        df_curve_out["impressions_GADS"].values, 
        res_GADS["alpha"], res_GADS["k"], res_GADS["beta"]
    )
    df_curve_out.to_csv("curvas_ajustadas_predicciones.csv", index=False)
    print("✓ Guardado: 'curvas_ajustadas_predicciones.csv'")


## 8. Resumen Ejecutivo


In [ ]:
if res_META is not None and res_GADS is not None:
    print("="*80)
    print("         RESUMEN: CURVAS DE RESPUESTA")
    print("="*80)
    
    print(f"\\n✅ SOLUCIÓN AL PROBLEMA DE MULTICOLINEALIDAD:")
    print("  • Antes: clicks + impressions → efecto repartido → curvas subestimadas")
    print("  • Ahora: SOLO impressions → efecto completo → curvas correctas")
    
    print(f"\\n📈 CURVA META:")
    print(f"  α = {res_META['alpha']:.4f} | k = {res_META['k']:,.0f} | β = {res_META['beta']:.4f}")
    print(f"  R² = {res_META['r2']:.3f}")
    
    print(f"\\n📈 CURVA GADS:")
    print(f"  α = {res_GADS['alpha']:.4f} | k = {res_GADS['k']:,.0f} | β = {res_GADS['beta']:.4f}")
    print(f"  R² = {res_GADS['r2']:.3f}")
    
    print(f"\\n💡 INTERPRETACIÓN:")
    if res_META['r2'] > 0.3 and res_GADS['r2'] > 0.3:
        print("  ✅ Las curvas tienen buen ajuste (R² > 0.3)")
        print("  ✅ Listas para optimización de presupuesto")
    else:
        print("  ⚠️ R² relativamente bajo → considerar:")
        print("     - Efectos por empresa (heterogeneidad)")
        print("     - Variables adicionales (estacionalidad, competencia)")
        print("     - Lags temporales en el efecto")
    
    print(f"\\n🎯 PRÓXIMOS PASOS:")
    print("  1. Validar curvas por empresa (si hay suficientes datos)")
    print("  2. Implementar optimizador de presupuesto usando estas curvas")
    print("  3. Simular escenarios de inversión")
    print("="*80)
else:
    print("\\n⚠️ No se pudieron generar curvas completas")
